## Materials and glass colors

This example shows how to:
   - use / modify predefined materials
   - manage colors of the transparent, glass-like material
   - setup range of path segments per traced ray appropriate to transparent materials in the scene
   - setup light shading and tone mapping for best caustics or fast convergence
   
Tkinter GUI window is launched from the notebook in this example. This allows re-running code cells and see results without scrolling back to the figure.

In [1]:
import numpy as np
from plotoptix import TkOptiX
from plotoptix.materials import m_clear_glass, m_mirror, m_plastic  # predefined materials
from plotoptix.utils import map_to_colors  # map variable to matplotlib color map

Make some data first:

In [2]:
n = 6
a = 8
s = a / n
xyz = np.mgrid[0:a:s, 0:a:s, 0:a:s].reshape(3,-1).T
rnd = np.random.uniform(0, 1, xyz.shape[0])

# most cubes with the standard material:
xyz_c_diffuse = xyz[((xyz[:,0] == 0) | (xyz[:,1] == 0) | (xyz[:,2] == 0)) & (rnd < 0.7)]
# the rest made of mirror walls:
xyz_c_mirror = xyz[((xyz[:,0] == 0) | (xyz[:,1] == 0) | (xyz[:,2] == 0)) & (rnd >= 0.7)]

# most particles made of glass:
xyz_p_glass = xyz[(xyz[:,0] > 0) & (xyz[:,1] > 0) & (xyz[:,2] > 0) & (rnd < 0.4)]
# some plastic-like particles:
xyz_p_plastic = xyz[(xyz[:,0] > 0) & (xyz[:,1] > 0) & (xyz[:,2] > 0) & (rnd > 0.9)]

Setup the raytracer using Tkinter GUI as the output target:

In [3]:
optix = TkOptiX(start_now=False) # no need to open the window yet
optix.set_param(min_accumulation_step=4,     # set more accumulation frames
                max_accumulation_frames=500) # to get rid of the noise.

**Note 1:** *transparent* and *reflective* materials may need increased number of traced ray segments, depending on the scene complexity. Default minimum is 2 segments per ray and maximum is 6 (russian roulette is applied between min and max to stop the ray). Default value is fine for opaque materials, but it may cut the ray too early when it is crossing several transparent objects.

Let's make the segments range longer:

In [4]:
optix.set_uint("path_seg_range", 15, 30)

Only *diffuse* material is available by default. Other materials need to be configured before using:

In [5]:
optix.setup_material("glass", m_clear_glass)
optix.setup_material("plastic", m_plastic)
optix.setup_material("mirror", m_mirror)

**Note 2:** range of color RGB components of opaque materials is <0; 1>, and it tells how much light for each component is scattered of the surface. The meaning of color in *transparent* materials is attenuation lenght of each component, and the range is <0; inf). So, if a glass object is to be transparent, its RGB values should be much higher than its dimensions.

That is why the color of particles in the cell below is set to 10 (meaning clear, neutral color with rgb=[10, 10, 10]).

In [6]:
optix.set_data("particles_g", pos=xyz_p_glass + np.array([0.5*s, 0.5*s, 0.5*s]), r=0.4*s,
               geom="ParticleSet",     # set the geometry of data points to particles
               mat="glass",            # use the glass material
               c=10)                   # and set the color to transparent, neutral
optix.set_data("particles_p", pos=xyz_p_plastic + np.array([0.5*s, 0.5*s, 0.5*s]), r=0.4*s,
               geom="ParticleSet",     # set geometry of to particles
               mat="plastic",          # slightly reflective, plastic look
               c=0.95)                 # white color

optix.set_data("cubes_d", pos=xyz_c_diffuse, u=[0.9*s, 0, 0], v=[0, 0.9*s, 0], w=[0, 0, 0.9*s],
               geom="Parallelepipeds", # cubes, actually default geometry
               mat="diffuse",          # opaque, mat, default
               c=0.95)                 # white color

optix.set_data("cubes_m", pos=xyz_c_mirror, u=[0.9*s, 0, 0], v=[0, 0.9*s, 0], w=[0, 0, 0.9*s],
               geom="Parallelepipeds", # cubes, same default geometry
               mat="mirror")           # 100% reflective mirror, no color

Setup a good point of view, and set background/ambient colors to zero:

In [7]:
optix.setup_camera("cam1", eye=[20, 10, 10], target=[0.5*a, 0.4*a, 0.5*a], fov=35)
optix.set_background(0)
optix.set_ambient(0)

Use "Hard" light shading for the best *caustics*. Use default "Soft" light shading for *fast convergence*. Always set light shading mode before adding lights.

In [8]:
optix.set_light_shading("Hard")
#optix.set_light_shading("Soft") # default light shading mode

Adjust exposure and gamma correction to improve brightness and contrast in the scene, use AI denoiser to improve image quality before accumulation converges. Note: use *Gamma* postprocessing OR *setup denoiser* - denoiser includes gamma correction.

In [9]:
optix.set_float("tonemap_exposure", 0.5)
optix.set_float("tonemap_igamma", 1 / 2.2) # note: inverse gamma is provided

optix.add_postproc("Gamma") # apply gamma correction postprocessing stage, or
#optix.setup_denoiser()      # use AI denoiser (exposure and gamma are applied as well)

In [10]:
optix.setup_light("light1", pos=[5, 20, 5], color=10*np.array([1.0, 0.97, 0.7]), radius=4)
optix.setup_light("light2", pos=[7, 7, 20], color=15*np.array([0.7, 0.85, 1.0]), radius=3)
optix.setup_light("light3", pos=[15, 2, 2], color=15, radius=1)

OK, finally open the GUI.

In [11]:
optix.start()

Wait for the raytracing completed, it is worth looking at the image without the noise.

---

Cells below, before closing the GUI, allow to update several scene parameters. You can run these cells multiple times.


Display glass parameters, note constant refraction index for R, G, and B components:

In [12]:
print(m_clear_glass)

{'ClosestHitPrograms': ['0::path_tracing_materials.ptx::glass_closest_hit'], 'AnyHitPrograms': ['1::path_tracing_materials.ptx::any_hit'], 'VarInt': {'material_flags': 12}, 'VarFloat': {'radiation_length': 0.0, 'vol_scattering': 1.0, 'light_emission': 0.0}, 'VarFloat3': {'refraction_index': [1.4, 1.4, 1.4], 'surface_albedo': [1.0, 1.0, 1.0]}}


Change refraction index, causing light dispersion. Wait for a few frames until green bias disappears:

In [13]:
m_clear_glass["VarFloat3"]["refraction_index"] = [1.4, 1.44, 1.5]
optix.update_material("glass", m_clear_glass, refresh=True)

See, how neutral colors can affect the transparency:

In [14]:
optix.update_data("particles_g", c = 0.5 * xyz_p_glass[:,1])

Use matplotlib colors, but scaled x10 to the range resulting with transparent glass:

In [15]:
optix.update_data("particles_g", c = 10 * map_to_colors(xyz_p_glass[:,1], "jet"))

Change lighting:

In [16]:
optix.update_light("light1", color=np.array([0.9, 0.7, 0]), radius=7)
optix.update_light("light2", pos=[15, 15, 15], color=10*np.array([0.2, 0.5, 1.0]), radius=3)

Min/max number of path segments per ray:

In [17]:
optix.set_uint("path_seg_range", 2, 4, refresh=True) # try finding min values for this scene

---
Close the GUI (or use "x" in the window).

In [18]:
optix.close()